# 第 9 章：錯誤處理

本章節詳細說明 Python 的例外處理機制，包含 try-except 語法、拋出例外、自訂例外類別，以及最佳實踐。

---

## 9.1 try-except（對應 try-catch）

### JavaScript vs Python 語法對照

```javascript
// JavaScript
try {
    // 可能出錯的程式碼
    riskyOperation();
} catch (error) {
    // 處理錯誤
    console.error(error.message);
} finally {
    // 一定會執行
    cleanup();
}
```

```python
# Python
try:
    # 可能出錯的程式碼
    risky_operation()
except Exception as e:
    # 處理錯誤
    print(f"錯誤：{e}")
finally:
    # 一定會執行
    cleanup()
```

### 基本 try-except 結構

In [ ]:
# 基本用法
try:
    result = 10 / 0
except ZeroDivisionError:
    print("不能除以零！")

In [ ]:
# 捕捉例外物件
try:
    result = 10 / 0
except ZeroDivisionError as e:
    print(f"錯誤類型：{type(e).__name__}")
    print(f"錯誤訊息：{e}")

### 捕捉多種例外

In [ ]:
# 方式 1：多個 except 區塊
def safe_divide(a, b):
    try:
        # 嘗試轉換並計算
        a = float(a)
        b = float(b)
        result = a / b
    except ValueError:
        print("請輸入有效的數字！")
        return None
    except ZeroDivisionError:
        print("不能除以零！")
        return None
    return result

print(safe_divide("abc", 2))  # ValueError
print(safe_divide(10, 0))      # ZeroDivisionError
print(safe_divide(10, 2))      # 正常

In [ ]:
# 方式 2：同時捕捉多種例外（相同處理方式）
def safe_divide_v2(a, b):
    try:
        return float(a) / float(b)
    except (ValueError, ZeroDivisionError) as e:
        print(f"輸入錯誤：{e}")
        return None

print(safe_divide_v2("abc", 2))
print(safe_divide_v2(10, 0))

In [ ]:
# 方式 3：分別處理後捕捉所有其他例外
def risky_operation():
    import random
    choice = random.choice([1, 2, 3])
    if choice == 1:
        raise ValueError("值錯誤")
    elif choice == 2:
        raise TypeError("型別錯誤")
    else:
        raise RuntimeError("執行時錯誤")

try:
    risky_operation()
except ValueError:
    print("處理 ValueError")
except TypeError:
    print("處理 TypeError")
except Exception as e:
    # 捕捉所有其他例外
    print(f"未預期的錯誤：{type(e).__name__}: {e}")

### else 子句

`else` 在沒有發生例外時執行（這是 Python 特有的功能，JavaScript 沒有對應語法）：

In [ ]:
def divide_with_else(a, b):
    try:
        result = a / b
    except ZeroDivisionError:
        print("不能除以零！")
        return None
    else:
        # 只有在沒有例外時才執行
        print(f"計算成功！")
        return result

print("結果:", divide_with_else(10, 2))
print("---")
print("結果:", divide_with_else(10, 0))

### finally 子句

`finally` 無論是否發生例外都會執行：

In [ ]:
def operation_with_finally(success=True):
    try:
        print("執行操作...")
        if not success:
            raise ValueError("操作失敗")
        print("操作完成")
        return "成功"
    except ValueError as e:
        print(f"捕捉到錯誤：{e}")
        return "失敗"
    finally:
        # 無論如何都會執行
        print("清理資源...（finally）")

print(f"\n成功案例: {operation_with_finally(True)}")
print("---")
print(f"\n失敗案例: {operation_with_finally(False)}")

In [ ]:
# finally 即使有 return 也會執行
def test_finally_with_return():
    try:
        print("try 區塊")
        return "try 的回傳值"
    finally:
        print("finally 區塊（仍然執行）")

result = test_finally_with_return()
print(f"函式回傳：{result}")

### 完整的 try 語句結構

In [ ]:
def complete_try_example(value):
    """展示完整的 try-except-else-finally 結構"""
    try:
        # 嘗試執行的程式碼
        result = 100 / value
    except ZeroDivisionError as e:
        # 處理特定錯誤
        print(f"[except] 除以零錯誤：{e}")
        result = None
    except TypeError as e:
        # 處理其他錯誤
        print(f"[except] 型別錯誤：{e}")
        result = None
    else:
        # 沒有錯誤時執行
        print(f"[else] 計算成功，結果 = {result}")
    finally:
        # 無論如何都執行
        print("[finally] 清理完成")
    
    return result

print("=== 正常情況 ===")
complete_try_example(10)

print("\n=== 除以零 ===")
complete_try_example(0)

print("\n=== 型別錯誤 ===")
complete_try_example("string")

### 巢狀的 try-except

In [ ]:
# 巢狀 try-except 示範
try:
    try:
        result = int("not a number")
    except ValueError:
        print("內層捕捉到 ValueError")
        raise  # 重新拋出，讓外層處理
except ValueError:
    print("外層捕捉到 ValueError")

---

## 9.2 拋出例外

### raise 語句

In [ ]:
# 拋出例外
def validate_positive(value):
    if value < 0:
        raise ValueError("值不能為負數")
    return value

try:
    validate_positive(-5)
except ValueError as e:
    print(f"驗證失敗：{e}")

In [ ]:
# 拋出帶有詳細訊息的例外
def check_type(value, expected_type):
    if not isinstance(value, expected_type):
        raise TypeError(
            f"期望 {expected_type.__name__}，但收到 {type(value).__name__}"
        )
    return value

try:
    check_type("hello", int)
except TypeError as e:
    print(f"型別檢查失敗：{e}")

### 條件式拋出例外

In [ ]:
def set_age(age):
    """設定年齡，包含完整驗證"""
    if not isinstance(age, int):
        raise TypeError(f"age 必須是整數，收到 {type(age).__name__}")
    if age < 0:
        raise ValueError("age 不能為負數")
    if age > 150:
        raise ValueError("age 不能超過 150")
    return age

# 測試各種錯誤情況
test_cases = [25, -5, 200, "thirty", 3.5]

for test in test_cases:
    try:
        result = set_age(test)
        print(f"set_age({test!r}) = {result}")
    except (TypeError, ValueError) as e:
        print(f"set_age({test!r}) 錯誤：{e}")

### 重新拋出例外

In [ ]:
# 使用 raise 重新拋出當前例外
def process_data(data):
    try:
        result = int(data)
        return result * 2
    except ValueError as e:
        print(f"記錄錯誤：{e}")
        raise  # 重新拋出原始例外

try:
    process_data("not a number")
except ValueError as e:
    print(f"外層捕捉到：{e}")

In [ ]:
import json

# 拋出新例外並保留原始例外資訊（使用 from）
def parse_config(json_str):
    try:
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        # 使用 from e 保留原始例外
        raise ValueError("設定檔格式錯誤") from e

try:
    parse_config("{invalid json}")
except ValueError as e:
    print(f"錯誤：{e}")
    print(f"原因：{e.__cause__}")

In [ ]:
# 使用 from None 隱藏原始例外
def parse_config_clean(json_str):
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        # 使用 from None 隱藏原始例外細節
        raise ValueError("設定檔格式錯誤") from None

try:
    parse_config_clean("{invalid json}")
except ValueError as e:
    print(f"錯誤：{e}")
    print(f"原因：{e.__cause__}")  # None

### assert 斷言

In [ ]:
# assert 用於開發期間的檢查
def calculate_average(numbers):
    assert len(numbers) > 0, "列表不能為空"
    assert all(isinstance(n, (int, float)) for n in numbers), "所有元素必須是數字"
    return sum(numbers) / len(numbers)

# 正常使用
print(calculate_average([1, 2, 3, 4, 5]))

# 觸發 assert
try:
    calculate_average([])
except AssertionError as e:
    print(f"斷言失敗：{e}")

#### assert 的注意事項

```python
# assert 在 python -O（優化模式）下會被忽略
# 所以不要用 assert 做重要的驗證

# 正確用法：開發期間的不變條件檢查
def binary_search(arr, target):
    assert arr == sorted(arr), "陣列必須已排序"
    # ...

# 錯誤用法：使用者輸入驗證（應該用 if + raise）
def set_age(age):
    # 不要這樣做！
    # assert age >= 0, "age 不能為負數"

    # 應該這樣做
    if age < 0:
        raise ValueError("age 不能為負數")
```

---

## 9.3 自訂例外

### 基本自訂例外

In [ ]:
# 最簡單的自訂例外
class MyError(Exception):
    """自訂例外類別"""
    pass

# 使用
try:
    raise MyError("發生自訂錯誤")
except MyError as e:
    print(f"捕捉到 MyError：{e}")

### 帶有額外資訊的例外

In [ ]:
class ValidationError(Exception):
    """驗證錯誤"""

    def __init__(self, message, field=None, value=None):
        super().__init__(message)
        self.field = field
        self.value = value
        self.message = message

    def __str__(self):
        if self.field:
            return f"{self.field}: {self.message} (收到: {self.value!r})"
        return self.message


# 使用
def validate_email(email):
    if "@" not in email:
        raise ValidationError(
            "無效的 email 格式",
            field="email",
            value=email
        )
    return email


try:
    validate_email("invalid-email")
except ValidationError as e:
    print(f"驗證失敗：{e}")
    print(f"欄位：{e.field}")
    print(f"值：{e.value}")

### 例外階層

In [ ]:
# 定義例外階層
class AppError(Exception):
    """應用程式基礎例外"""
    pass

class ConfigError(AppError):
    """設定相關錯誤"""
    pass

class DatabaseError(AppError):
    """資料庫相關錯誤"""
    pass

class ConnectionError(DatabaseError):
    """資料庫連線錯誤"""
    pass

class QueryError(DatabaseError):
    """資料庫查詢錯誤"""
    pass

# 使用例外階層
def simulate_db_operation(operation):
    if operation == "connect":
        raise ConnectionError("無法連接資料庫")
    elif operation == "query":
        raise QueryError("查詢語法錯誤")
    elif operation == "config":
        raise ConfigError("設定檔遺失")

# 可以用父類別捕捉所有子類別的例外
for op in ["connect", "query", "config"]:
    try:
        simulate_db_operation(op)
    except ConnectionError:
        print("連線失敗，嘗試重新連線...")
    except DatabaseError:
        print("資料庫錯誤")
    except AppError:
        print("應用程式錯誤")

### 實際應用範例：HTTP API 例外

In [ ]:
# HTTP API 例外階層
class APIError(Exception):
    """API 基礎例外"""

    def __init__(self, message, status_code=500, details=None):
        super().__init__(message)
        self.message = message
        self.status_code = status_code
        self.details = details or {}

    def to_dict(self):
        return {
            "error": self.__class__.__name__,
            "message": self.message,
            "status_code": self.status_code,
            "details": self.details
        }


class BadRequestError(APIError):
    """400 Bad Request"""
    def __init__(self, message, details=None):
        super().__init__(message, status_code=400, details=details)


class NotFoundError(APIError):
    """404 Not Found"""
    def __init__(self, resource, resource_id):
        message = f"{resource} with id '{resource_id}' not found"
        super().__init__(message, status_code=404)
        self.resource = resource
        self.resource_id = resource_id


class UnauthorizedError(APIError):
    """401 Unauthorized"""
    def __init__(self, message="Authentication required"):
        super().__init__(message, status_code=401)

In [ ]:
# 使用 API 例外
def get_user(user_id, users_db):
    if user_id not in users_db:
        raise NotFoundError("User", user_id)
    return users_db[user_id]

# 模擬資料庫
users = {
    1: {"name": "Alice", "email": "alice@example.com"},
    2: {"name": "Bob", "email": "bob@example.com"}
}

# 測試
for uid in [1, 99]:
    try:
        user = get_user(uid, users)
        print(f"找到用戶：{user}")
    except APIError as e:
        print(f"API 錯誤：{e.to_dict()}")

---

## 9.4 常見內建例外

### 例外階層圖

```
BaseException
├── SystemExit                 # sys.exit() 呼叫
├── KeyboardInterrupt          # Ctrl+C 中斷
├── GeneratorExit              # 生成器關閉
└── Exception                  # 所有一般例外的基礎類別
    ├── StopIteration          # 迭代結束
    ├── ArithmeticError        # 算術錯誤基礎類別
    │   ├── ZeroDivisionError  # 除以零
    │   ├── OverflowError      # 數值溢位
    │   └── FloatingPointError # 浮點數錯誤
    ├── AssertionError         # assert 失敗
    ├── AttributeError         # 屬性不存在
    ├── ImportError            # 匯入失敗
    │   └── ModuleNotFoundError # 模組找不到
    ├── LookupError            # 查找錯誤基礎類別
    │   ├── IndexError         # 索引超出範圍
    │   └── KeyError           # 字典鍵不存在
    ├── NameError              # 名稱未定義
    ├── OSError                # 作業系統錯誤
    │   ├── FileNotFoundError  # 檔案不存在
    │   ├── PermissionError    # 權限不足
    │   └── TimeoutError       # 操作超時
    ├── RuntimeError           # 執行時錯誤
    │   ├── NotImplementedError # 未實作
    │   └── RecursionError     # 遞迴過深
    ├── TypeError              # 型別錯誤
    └── ValueError             # 值錯誤
```

### 常見例外示範

In [ ]:
# ValueError - 值不正確
try:
    num = int("not a number")
except ValueError as e:
    print(f"ValueError: {e}")

In [ ]:
# TypeError - 型別不正確
try:
    result = "hello" + 123
except TypeError as e:
    print(f"TypeError: {e}")

In [ ]:
# KeyError - 字典鍵不存在
try:
    data = {"name": "Alice"}
    print(data["age"])
except KeyError as e:
    print(f"KeyError: 鍵 {e} 不存在")

In [ ]:
# IndexError - 索引超出範圍
try:
    items = [1, 2, 3]
    print(items[10])
except IndexError as e:
    print(f"IndexError: {e}")

In [ ]:
# AttributeError - 屬性不存在
try:
    "hello".non_existent_method()
except AttributeError as e:
    print(f"AttributeError: {e}")

In [ ]:
# ZeroDivisionError - 除以零
try:
    result = 10 / 0
except ZeroDivisionError as e:
    print(f"ZeroDivisionError: {e}")

In [ ]:
# FileNotFoundError - 檔案不存在
try:
    with open("non_existent_file_12345.txt") as f:
        content = f.read()
except FileNotFoundError as e:
    print(f"FileNotFoundError: {e}")

In [ ]:
# ModuleNotFoundError - 模組不存在
try:
    import non_existent_module_12345
except ModuleNotFoundError as e:
    print(f"ModuleNotFoundError: {e}")

In [ ]:
# RecursionError - 遞迴過深
def infinite_recursion():
    return infinite_recursion()

try:
    infinite_recursion()
except RecursionError as e:
    print(f"RecursionError: {e}")

### 不應該捕捉的例外

```python
# 不要捕捉這些例外（除非有特殊需求）

# SystemExit - sys.exit() 會觸發
# 捕捉它會阻止程式正常退出

# KeyboardInterrupt - Ctrl+C 會觸發
# 捕捉它會讓使用者無法中斷程式

# 如果需要清理資源，使用 finally
try:
    while True:
        do_work()
except KeyboardInterrupt:
    print("程式被中斷")
finally:
    cleanup()  # 清理資源
```

---

## 9.5 Context Manager 與例外處理

### with 語句自動處理例外

In [ ]:
# with 語句會自動處理資源釋放
# 即使發生例外，檔案也會被關閉

# 使用 with（推薦）
try:
    with open("test_file.txt", "w") as f:
        f.write("Hello, World!")
        # 即使這裡發生例外，檔案也會正確關閉
    print("檔案已寫入並關閉")
except IOError as e:
    print(f"IO 錯誤：{e}")

In [ ]:
# 等價於手動處理
f = None
try:
    f = open("test_file.txt", "r")
    content = f.read()
    print(f"檔案內容：{content}")
finally:
    if f:
        f.close()
        print("檔案已關閉")

In [ ]:
# 多個 context manager
try:
    with open("input.txt", "w") as fin:
        fin.write("Source content")
    
    with open("input.txt", "r") as fin, open("output.txt", "w") as fout:
        fout.write(fin.read())
    
    print("檔案複製完成")
except IOError as e:
    print(f"IO 錯誤：{e}")

### contextlib 模組

In [ ]:
from contextlib import contextmanager
import time

# 使用 @contextmanager 建立 context manager
@contextmanager
def timer(name):
    start = time.time()
    try:
        yield  # 這裡執行 with 區塊的程式碼
    finally:
        elapsed = time.time() - start
        print(f"{name} 耗時：{elapsed:.4f} 秒")

# 使用計時器
with timer("資料處理"):
    # 模擬耗時操作
    total = sum(range(1000000))
    print(f"計算結果：{total}")

In [ ]:
from contextlib import suppress
import os

# suppress - 忽略特定例外

# 傳統寫法
try:
    os.remove("non_existent_file.txt")
except FileNotFoundError:
    pass  # 忽略檔案不存在的錯誤

# 使用 suppress（更簡潔）
with suppress(FileNotFoundError):
    os.remove("non_existent_file.txt")

print("suppress 完成，不會有錯誤")

In [ ]:
# suppress 多個例外
from contextlib import suppress

with suppress(FileNotFoundError, PermissionError):
    os.remove("another_non_existent_file.txt")

print("多個例外的 suppress 完成")

### 自訂 Context Manager 處理例外

In [ ]:
class TransactionManager:
    """模擬資料庫交易管理"""
    
    def __init__(self, name):
        self.name = name
    
    def __enter__(self):
        print(f"[{self.name}] 開始交易")
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is not None:
            # 發生例外，回滾
            print(f"[{self.name}] 發生錯誤：{exc_val}，回滾交易")
            # 回傳 False 會繼續傳播例外
            # 回傳 True 會抑制例外
            return False
        else:
            # 沒有例外，提交
            print(f"[{self.name}] 提交交易")
            return True
    
    def execute(self, sql):
        print(f"[{self.name}] 執行：{sql}")


# 成功的交易
print("=== 成功的交易 ===")
with TransactionManager("TX1") as tx:
    tx.execute("INSERT INTO users VALUES (1, 'Alice')")
    tx.execute("UPDATE accounts SET balance = 100")

print("\n=== 失敗的交易 ===")
try:
    with TransactionManager("TX2") as tx:
        tx.execute("INSERT INTO users VALUES (2, 'Bob')")
        raise ValueError("模擬錯誤")
        tx.execute("這行不會執行")
except ValueError:
    print("外層捕捉到例外")

---

## 9.6 例外處理最佳實踐

### 1. 捕捉具體的例外

In [ ]:
# 不好：捕捉所有例外
def bad_example(data):
    try:
        return data["key"] / data["divisor"]
    except:  # 或 except Exception:
        pass  # 吞掉所有錯誤，難以除錯

print("bad_example 結果:", bad_example({"key": 10}))  # 返回 None，不知道為什麼

In [ ]:
# 好：捕捉具體的例外
def good_example(data):
    try:
        return data["key"] / data["divisor"]
    except KeyError as e:
        print(f"缺少必要的鍵：{e}")
        return None
    except ZeroDivisionError:
        print("除數不能為零")
        return None
    except TypeError as e:
        print(f"型別錯誤：{e}")
        return None

print("good_example 結果:", good_example({"key": 10}))  # 清楚知道問題

### 2. 不要吞掉例外

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("demo")

# 好：至少記錄例外
def logged_operation(x):
    try:
        return 10 / x
    except ZeroDivisionError:
        logger.exception("除法操作失敗")  # 記錄完整的堆疊追蹤
        raise  # 重新拋出

try:
    logged_operation(0)
except ZeroDivisionError:
    print("在外層處理例外")

### 3. 使用 EAFP 原則

Python 傾向使用「請求寬恕比請求許可更容易」(EAFP - Easier to Ask Forgiveness than Permission) 的風格：

In [ ]:
# LBYL (Look Before You Leap) - 先檢查
# 較不 Pythonic
def lbyl_get_value(dictionary, key, default=None):
    if key in dictionary:
        return dictionary[key]
    else:
        return default

# EAFP (Easier to Ask Forgiveness than Permission) - 直接做
# 較 Pythonic
def eafp_get_value(dictionary, key, default=None):
    try:
        return dictionary[key]
    except KeyError:
        return default

# 最 Pythonic：使用內建方法
def pythonic_get_value(dictionary, key, default=None):
    return dictionary.get(key, default)

# 測試
data = {"name": "Alice"}
print(f"LBYL: {lbyl_get_value(data, 'age', 0)}")
print(f"EAFP: {eafp_get_value(data, 'age', 0)}")
print(f"Pythonic: {pythonic_get_value(data, 'age', 0)}")

### 4. 保持 try 區塊精簡

In [ ]:
# 不好：try 區塊太大
def bad_processing(filename):
    try:
        with open(filename) as f:
            data = f.read()
        processed = data.upper()
        result = len(processed)
        return result
    except Exception as e:
        print(f"錯誤：{e}")  # 不知道是哪一步出錯
        return None

# 好：精確捕捉
def good_processing(filename):
    # 分開處理不同的錯誤
    try:
        with open(filename) as f:
            data = f.read()
    except FileNotFoundError:
        print(f"檔案不存在：{filename}")
        return None
    except PermissionError:
        print(f"沒有讀取權限：{filename}")
        return None
    
    # 後續處理不需要 try-except
    processed = data.upper()
    result = len(processed)
    return result

print(good_processing("non_existent.txt"))

### 5. 提供有用的錯誤訊息

In [ ]:
# 不好：訊息不明確
def bad_validate(email):
    if "@" not in email:
        raise ValueError("錯誤")

# 好：訊息包含有用資訊
def good_validate(email):
    if "@" not in email:
        raise ValueError(
            f"無效的 email 格式：'{email}'，"
            f"email 必須包含 '@' 字元"
        )

try:
    good_validate("invalid-email")
except ValueError as e:
    print(f"驗證錯誤：{e}")

### 6. 使用例外而非回傳錯誤碼

In [ ]:
# 不好：使用錯誤碼（類似 Go 或 C 的風格）
def divide_with_error_code(a, b):
    if b == 0:
        return None, "除數不能為零"
    return a / b, None

result, error = divide_with_error_code(10, 0)
if error:
    print(f"錯誤碼方式：{error}")

# 好：使用例外（Pythonic 風格）
def divide_with_exception(a, b):
    if b == 0:
        raise ZeroDivisionError("除數不能為零")
    return a / b

try:
    result = divide_with_exception(10, 0)
except ZeroDivisionError as e:
    print(f"例外方式：{e}")

### 7. 清理資源使用 finally 或 with

In [ ]:
# 使用 finally
def process_with_finally(filename):
    f = None
    try:
        f = open(filename, "w")
        f.write("test")
        return "成功"
    except IOError as e:
        return f"失敗：{e}"
    finally:
        if f:
            f.close()
            print("finally: 檔案已關閉")

# 更好：使用 with
def process_with_context(filename):
    try:
        with open(filename, "w") as f:
            f.write("test")
        return "成功"
    except IOError as e:
        return f"失敗：{e}"

print(process_with_finally("test1.txt"))
print(process_with_context("test2.txt"))

---

## 9.7 除錯與追蹤

### 取得例外資訊

In [ ]:
import traceback
import sys

def level3():
    return 1 / 0

def level2():
    return level3()

def level1():
    return level2()

try:
    level1()
except ZeroDivisionError as e:
    # 例外類型和訊息
    print(f"類型：{type(e).__name__}")
    print(f"訊息：{e}")
    
    # 取得完整的堆疊追蹤
    print("\n堆疊追蹤：")
    traceback.print_exc()

In [ ]:
import traceback

# 取得堆疊追蹤字串
try:
    1 / 0
except ZeroDivisionError:
    tb_str = traceback.format_exc()
    print("堆疊追蹤字串：")
    print(tb_str)

### logging 與例外

In [ ]:
import logging

# 設定 logging
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger("example")

def risky_function():
    raise ValueError("測試錯誤")

try:
    risky_function()
except ValueError as e:
    # 記錄例外（包含堆疊追蹤）
    logger.exception("操作失敗")

---

## 練習題

### 練習 1：安全的字典存取

寫一個函式，安全地從巢狀字典取值：

In [ ]:
# 練習 1：你的程式碼
def safe_get(data, *keys, default=None):
    """
    安全地從巢狀字典取值
    
    >>> data = {"a": {"b": {"c": 1}}}
    >>> safe_get(data, "a", "b", "c")
    1
    >>> safe_get(data, "a", "x", "y", default=0)
    0
    """
    # 你的程式碼
    pass

In [ ]:
# 練習 1：參考解答
def safe_get(data, *keys, default=None):
    """
    安全地從巢狀字典取值
    """
    result = data
    for key in keys:
        try:
            result = result[key]
        except (KeyError, TypeError, IndexError):
            return default
    return result

# 測試
data = {"a": {"b": {"c": 1}}}
print(f"safe_get(data, 'a', 'b', 'c') = {safe_get(data, 'a', 'b', 'c')}")
print(f"safe_get(data, 'a', 'x', 'y', default=0) = {safe_get(data, 'a', 'x', 'y', default=0)}")
print(f"safe_get(data, 'a', 'b') = {safe_get(data, 'a', 'b')}")

### 練習 2：重試裝飾器

寫一個裝飾器，在函式失敗時自動重試：

In [ ]:
# 練習 2：你的程式碼
def retry(max_attempts=3, exceptions=(Exception,)):
    """
    重試裝飾器
    
    @retry(max_attempts=3, exceptions=(ConnectionError,))
    def fetch_data():
        ...
    """
    # 你的程式碼
    pass

In [ ]:
# 練習 2：參考解答
from functools import wraps
import time

def retry(max_attempts=3, exceptions=(Exception,), delay=0.1):
    """重試裝飾器"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            last_exception = None
            for attempt in range(1, max_attempts + 1):
                try:
                    return func(*args, **kwargs)
                except exceptions as e:
                    last_exception = e
                    print(f"嘗試 {attempt}/{max_attempts} 失敗：{e}")
                    if attempt < max_attempts:
                        time.sleep(delay)
            raise last_exception
        return wrapper
    return decorator

# 測試
call_count = 0

@retry(max_attempts=3, exceptions=(ValueError,))
def flaky_function():
    global call_count
    call_count += 1
    if call_count < 3:
        raise ValueError(f"第 {call_count} 次呼叫失敗")
    return "成功！"

try:
    result = flaky_function()
    print(f"結果：{result}")
except ValueError as e:
    print(f"最終失敗：{e}")

### 練習 3：自訂例外階層

為一個電商系統設計例外階層：

In [ ]:
# 練習 3：你的程式碼
# 設計以下例外類別：
# - OrderError (基礎)
#   - InvalidOrderError
#   - PaymentError
#     - InsufficientFundsError
#     - PaymentDeclinedError
#   - ShippingError
#     - AddressNotFoundError
#     - OutOfStockError


In [ ]:
# 練習 3：參考解答
class OrderError(Exception):
    """訂單基礎例外"""
    def __init__(self, message, order_id=None):
        super().__init__(message)
        self.order_id = order_id

class InvalidOrderError(OrderError):
    """無效訂單"""
    pass

class PaymentError(OrderError):
    """付款錯誤"""
    def __init__(self, message, order_id=None, amount=None):
        super().__init__(message, order_id)
        self.amount = amount

class InsufficientFundsError(PaymentError):
    """餘額不足"""
    def __init__(self, order_id=None, amount=None, balance=None):
        message = f"餘額不足：需要 {amount}，但只有 {balance}"
        super().__init__(message, order_id, amount)
        self.balance = balance

class PaymentDeclinedError(PaymentError):
    """付款被拒絕"""
    def __init__(self, order_id=None, reason=None):
        message = f"付款被拒絕：{reason}"
        super().__init__(message, order_id)
        self.reason = reason

class ShippingError(OrderError):
    """運送錯誤"""
    pass

class AddressNotFoundError(ShippingError):
    """地址找不到"""
    def __init__(self, order_id=None, address=None):
        message = f"無效地址：{address}"
        super().__init__(message, order_id)
        self.address = address

class OutOfStockError(ShippingError):
    """缺貨"""
    def __init__(self, order_id=None, product=None, requested=None, available=None):
        message = f"商品 {product} 缺貨：要求 {requested}，庫存 {available}"
        super().__init__(message, order_id)
        self.product = product
        self.requested = requested
        self.available = available


# 測試例外階層
def process_order(order_type):
    if order_type == "insufficient":
        raise InsufficientFundsError(order_id="ORD001", amount=100, balance=50)
    elif order_type == "declined":
        raise PaymentDeclinedError(order_id="ORD002", reason="卡片過期")
    elif order_type == "stock":
        raise OutOfStockError(order_id="ORD003", product="iPhone", requested=5, available=2)

for order_type in ["insufficient", "declined", "stock"]:
    try:
        process_order(order_type)
    except PaymentError as e:
        print(f"付款問題：{e}")
    except ShippingError as e:
        print(f"運送問題：{e}")
    except OrderError as e:
        print(f"訂單問題：{e}")
    print()

---

## 小結

### JavaScript vs Python 例外處理對照

| 特性 | JavaScript | Python |
|------|------------|--------|
| 捕捉例外 | `try...catch` | `try...except` |
| 最終執行 | `finally` | `finally` |
| 拋出例外 | `throw new Error()` | `raise Exception()` |
| 無例外時 | 無 | `else` |
| 例外類型 | `Error`, `TypeError`, ... | `Exception`, `TypeError`, ... |
| 自訂例外 | `class MyError extends Error` | `class MyError(Exception)` |

### try 語句結構

```python
try:
    # 嘗試執行
except SpecificError:
    # 處理特定錯誤
except (Error1, Error2) as e:
    # 處理多種錯誤
except Exception as e:
    # 處理所有其他錯誤
else:
    # 沒有錯誤時執行
finally:
    # 無論如何都執行
```

### 最佳實踐摘要

1. **捕捉具體的例外**，不要使用裸露的 `except:`
2. **不要吞掉例外**，至少要記錄
3. **使用 EAFP 原則**
4. **保持 try 區塊精簡**
5. **提供有用的錯誤訊息**
6. **使用例外而非錯誤碼**
7. **使用 with 語句管理資源**
8. **建立有意義的例外階層**

### 常見例外速查

| 例外 | 說明 | 常見情境 |
|------|------|----------|
| `ValueError` | 值不正確 | `int("abc")` |
| `TypeError` | 型別不正確 | `"a" + 1` |
| `KeyError` | 字典鍵不存在 | `d["missing"]` |
| `IndexError` | 索引超出範圍 | `lst[100]` |
| `AttributeError` | 屬性不存在 | `obj.missing` |
| `FileNotFoundError` | 檔案不存在 | `open("missing.txt")` |
| `ZeroDivisionError` | 除以零 | `10 / 0` |
| `ImportError` | 匯入失敗 | `import missing` |
| `RuntimeError` | 執行時錯誤 | 一般錯誤 |
| `AssertionError` | 斷言失敗 | `assert False` |